In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://prod.aufgabenpool.at/srdp/phpcode/search.php"


all=[]
#loop through a to z
for i in range(97, 123):
#post request with query=buchstabe as payload
    r = requests.post(url = URL, data = {'query': chr(i)})
    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")
    #get all <li> tags
    li = soup.find_all('li')

    
    #get "id" attribute of the <div> in <li> tag
    li_id = [i.find('div')['id'] for i in li]

    #get inner text of <li> tags
    li_text = [i.text for i in li]

    #remove text between and "<" until ">" 
    li_text = [i.split("<")[0] for i in li_text]

    #remove "(number)" at the end
    li_text = [i.split("(")[0] for i in li_text]

    #trim whitespaces
    li_text = [i.strip() for i in li_text]

    #combine id and text to json
    li_text = [{"id": li_id[i], "thema": li_text[i]} for i in range(len(li_id))]

    all += li_text
    print(len(all))
    time.sleep(0.2)

#remove duplicates
all = list({v['id']:v for v in all}.values())

#remove {"id": "Ooops ...", "thema": "Ihre Suche hat leider keine Treffer erzielt."}
all = [i for i in all if i['id'] != "Ooops ..."]

#save as json
with open('/content/drive/MyDrive/themen.json', 'w') as outfile:
    json.dump(all, outfile)


37
44
68
86
133
151
183
217
261
262
284
321
348
397
432
442
445
483
509
551
587
595
602
605
608
622


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://aufgabenpool.at/srdp/phpcode/lp_results.php"

#read themen.json
with open('/content/drive/MyDrive/themen.json') as json_file:
    data = json.load(json_file)

res=[]
for el in data:
    payload = {'id': el['id'], 'coll': ''}

    #post request
    r = requests.post(url = URL, data = payload)

    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")

    #extract all <li> tags
    li = soup.find_all('li')

    #pdf is <a> tag
    #images can be found with src attribute from <img> tag

    all_imgs = []
    for i in li:
        a_tag = i.find('a')
        if a_tag:
          temp= {'loesung': "https://aufgabenpool.at/srdp/"+a_tag['href']}
        else:
          print(i)
          continue
        #get string between last / and .pdf
        temp['name'] = i.find('a')['href'].split("/")[-1].split(".pdf")[0]
        #find all <img> tags
        imgs = i.find_all('img')
        for i in range(len(imgs)):
            if i == 0:
                temp['aufgabe_head'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                temp['teil'] = imgs[i]['src'].split("teil")[1].split("/")[0].upper()
                temp['id'] = imgs[i]['src'].split("/")[1].split("/")[0].upper()
                temp["thema"]=el['thema']
            elif i == 1:
                temp['aufgabe_body'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                break
        res.append(temp)
    print(len(res))
    time.sleep(10)

#save as json
with open('/content/drive/MyDrive/beispiele.json', 'w') as outfile:
    json.dump(res, outfile)



In [ ]:
!pip install pytesseract PyPDF2 pdf2image 


In [ ]:
!pip install easyocr

In [ ]:
!sudo apt-get install poppler-utils

In [16]:
!sudo apt-get update
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-deu tesseract-ocr-script-latn

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree     

restart runtime

In [ ]:
import json
import sys
sys.path.append('/usr/local/lib/python3.8/dist-packages')
import requests
from PIL import Image
from io import BytesIO
import pytesseract
import PyPDF2
from PIL import Image
import os
from pdf2image import convert_from_path
import easyocr

# Loop through the pages of the PDF
def find_right_page(pdf_reader, such_string):
    such_page=len(pdf_reader.pages)-1
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
    #if second time found such_string, break
        con=0
        con2=0
    #check count how often such_string in text, check if lösung - bewertungschlüssel on one page
        for i in range(len(text)):
            if text[i:i+len(such_string)] == such_string:
                con+=1
            #if second time found such_string, break
                if con == 2:
                    such_page = page_num
                    break
    #if not found, check if such_string in text, qst is aufgabe, 2nd läsung
        if such_string in text:
            such_page = page_num
            break
    return such_page

#open beispiele.json
with open('/content/drive/MyDrive/beispiele.json') as json_file:
    data = json.load(json_file)
reader = easyocr.Reader(['de'], gpu=False)
thema="Binomialverteilung"

#filter by thema
data = [i for i in data if i['thema'] == thema]

#if no thema folder make one
if not os.path.exists("/content/drive/MyDrive/"+thema):
    os.makedirs("/content/drive/MyDrive/"+thema)

    #if no thema folder make one
if not os.path.exists("/content/drive/MyDrive/pdfs"):
    os.makedirs("/content/drive/MyDrive/pdfs")


for bsp in data:
    aufgabe_name= bsp["name"]+"_"+bsp["id"]

    auf_dir="/content/drive/MyDrive/"+thema+"/"+aufgabe_name
    if os.path.exists(auf_dir+"/"+"loesung.jpeg"):
        continue
    
    #get "aufgabe_body"
    ab=bsp['aufgabe_body']

    #get ab as image and ocr image
    response = requests.get(ab)
    img = Image.open(BytesIO(response.content))
    #ocr image with easyocr
    
    result = reader.readtext(img)
    print(result)
    text = result[0][1]
    
    #download "loesung" as pdf
    loe=bsp['loesung']
    pdf_name= aufgabe_name+".pdf"
    #check if already downloaded
    if not os.path.exists("/content/drive/MyDrive/"+"pdfs/"+pdf_name):
        r = requests.get(loe, allow_redirects=True)
        open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, 'wb').write(r.content)

    #extract bsp letter + ")"
    aufgabe = text.split(")")[0].strip()


    pdf_file = open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, "rb")
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    #manually type in the aufgabe letter if aufgabe is longer than 5 char
    if len(aufgabe) > 5:

        #find text before aufgabe   
        for i in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            page_text = page.extract_text()
            if aufgabe[:12] in page_text:
                #get index of aufgabe
                index = page_text.index(aufgabe[:12])
                au=page_text[index-10:index]
                aufgabe=au.strip().split(" ")[-1][:-1]

    #make aufgabe_name directory in thema dir
    if not os.path.exists(auf_dir):
        os.makedirs(auf_dir)
    
    #download "aufgabe_head" as image
    ah=bsp['aufgabe_head']
    ah_name= bsp["id"]+"ah"
    ah_path= auf_dir+"/"+ah_name+".png"
    #check if already downloaded
    if not os.path.exists(ah_path):
        response = requests.get(ah)
        img = Image.open(BytesIO(response.content))
        img.save(ah_path)

    #download "aufgabe_body" as image
    ab=bsp['aufgabe_body']
    ab_name= bsp["id"]+"ab"
    ab_path= auf_dir+"/"+ab_name+".png"
    #check if already downloaded
    if not os.path.exists(ab_path):
        response = requests.get(ab)
        img = Image.open(BytesIO(response.content))
        img.save(ab_path)
    
    #extract solution

    such_string = aufgabe+"1)"
    print(such_string)
    such_page = None



    print(such_string)
    print(pdf_file)
    such_page = find_right_page(pdf_reader, such_string)


    if such_page is None:
        such_string = aufgabe+")"
        such_page = find_right_page(pdf_reader, such_string)

        

    if such_page is not None:
        #save page as image
        pages = convert_from_path("/content/drive/MyDrive/"+"pdfs/"+pdf_name, dpi=180)
        pages[such_page].save(auf_dir+"/"+bsp["id"]+"loesung.jpeg", 'JPEG')
        # Open the image
        im = Image.open(auf_dir+"/"+bsp["id"]+"loesung.jpeg")

        # Define the crop coordinates
        left = 0
        top = 0
        right = im.width
        bottom = im.height
        
        # Find the index of the desired string
        # Get the bounding box coordinates for each character in the string
        boxes = pytesseract.image_to_boxes(im, lang='deu')
        whole_string = ""
        ind=0
        for b in boxes.splitlines():
            whole_string += b.split(' ')[0]
            if such_string in whole_string and top == 0:
                ind=len(whole_string)-1
                 # Get the bounding box coordinates
                b = b.split(' ')
                top = im.height - int(b[2]) - 50
                if top <= 0:
                    top = 1
            #if such_string is two times in text, break
            if such_string in whole_string[ind+1:] and ind!=0:
                b = b.split(' ')
                pos2 = int(b[2])
                bottom = im.height - int(b[2]) - 40
                break

        # Crop the image
        cropped_im = im.crop((left, top, right, bottom))

        # Save the cropped image
        cropped_im.save(auf_dir+"/"+bsp["id"]+"loesung.jpeg")
    else:
        print(loe)
        
    pdf_file.close()





In [56]:
!pip install genanki

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 3.9 MB/s eta 0:00:00


In [63]:
import json
import genanki


#open beispiele.json
with open('/content/drive/MyDrive/beispiele.json') as json_file:
    data = json.load(json_file)

thema="Binomialverteilung"

my_deck = genanki.Deck(
    deck_id=12345, # a unique id for the deck
    name=thema+"_RDP_scraped") # the name of the deck

img=[]
#filter by thema
data = [i for i in data if i['thema'] == thema]
c=0
for bsp in data:
    c+=1
    aufgabe_name= bsp["name"]+"_"+bsp["id"]
    auf_dir="/content/drive/MyDrive/"+thema+"/"+aufgabe_name
    ah=bsp['aufgabe_head']
    ah_name= bsp["id"]+"ah"
    ah_path= auf_dir+"/"+ah_name+".png"
    ab=bsp['aufgabe_body']
    ab_name= bsp["id"]+"ab"
    ab_path= auf_dir+"/"+ab_name+".png"
    my_card = genanki.Note(
    model=genanki.Model(
        1380120064,
  'Example',
        fields=[
            {'name': 'Image1'},
            {'name': 'Image2'},
            
            {'name': 'AnswerImage'},
            {'name': 'Answer'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Image1}}<br>{{Image2}}',
                'afmt': '{{AnswerImage}}<br>{{Answer}}',
            },
        ]
    ),
    fields=['<img src="{}">'.format(ah_name+".png"), '<img src="{}">'.format(ab_name+".png"), '<img src="{}">'.format(bsp["id"]+"loesung.jpeg"), '<a href="{}">Link</a>'.format(bsp["loesung"])])

    img.append(ah_path)
    img.append(ab_path)
    img.append(auf_dir+"/"+bsp["id"]+"loesung.jpeg")
    my_deck.add_note(my_card)

my_package = genanki.Package(my_deck)
my_package.media_files = img

#create directory if not exists
if not os.path.exists("/content/drive/MyDrive/AM_RDP_Decks"):
    os.makedirs("/content/drive/MyDrive/AM_RDP_Decks")

my_package.write_to_file("/content/drive/MyDrive/AM_RDP_Decks/"+thema+'.apkg')



